# RNN (recurrent neural network)

## Syntetic dataset

In [1]:
import numpy as np

# Parameters
num_samples = 10
sequence_length = 4

# Generate a synthetic time series dataset
def generate_sequence(start_value, length):
    sequence = [start_value]
    for _ in range(length - 1):
        next_value = sequence[-1] + np.random.normal(0, 0.1)
        sequence.append(next_value)
    return sequence

# Create the dataset
X = []
y = []

for _ in range(num_samples):
    start_value = np.random.uniform(-1, 1)
    sequence = generate_sequence(start_value, sequence_length)
    target_value = sequence[-1] + np.random.normal(0, 0.1)

    X.append(sequence[:-1])  # Input sequence
    y.append(target_value)    # Target value to predict

X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

(10, 3)
(10,)


In [2]:
import tensorflow as tf


model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=64, activation='tanh', input_shape=(None, 1)),
    tf.keras.layers.Dense(units=1)
])

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=50)

2023-10-26 08:44:15.295948: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50


2023-10-26 08:44:16.847303: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 436ms/step - loss: 1.3304
Epoch 2/50
1/1 [==============================] - 0s 4ms/step - loss: 1.0779
Epoch 3/50
1/1 [==============================] - 0s 4ms/step - loss: 0.8544
Epoch 4/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6603
Epoch 5/50
1/1 [==============================] - 0s 4ms/step - loss: 0.4952
Epoch 6/50
1/1 [==============================] - 0s 4ms/step - loss: 0.3584
Epoch 7/50
1/1 [==============================] - 0s 4ms/step - loss: 0.2493
Epoch 8/50
1/1 [==============================] - 0s 4ms/step - loss: 0.1670
Epoch 9/50
1/1 [==============================] - 0s 4ms/step - loss: 0.1105
Epoch 10/50
1/1 [==============================] - 0s 4ms/step - loss: 0.0779
Epoch 11/50
1/1 [==============================] - 0s 4ms/step - loss: 0.0663
Epoch 12/50
1/1 [==============================] - 0s 4ms/step - loss: 0.0713
Epoch 13/50
1/1 [==============================] - 0s 4ms/step - loss: 0.0876
Epoc

In [3]:
model.evaluate(X, y)

1/1 [==============================] - 2s 2s/step - loss: 0.0599


0.05989203602075577

## Pytorch

https://github.com/gabrielloye/RNN-walkthrough/blob/master/main.ipynb

In [7]:
import torch
from torch import nn

import numpy as np

In [8]:
text = ['hey how are you','good i am fine','have a nice day']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

In [9]:
print(char2int)

{'y': 0, 'i': 1, 'n': 2, 'e': 3, 'a': 4, 'u': 5, 'm': 6, 'w': 7, 'g': 8, 'v': 9, 'c': 10, 'r': 11, 'f': 12, ' ': 13, 'o': 14, 'h': 15, 'd': 16}


In [10]:
maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))

The longest string has 15 characters


In [11]:


# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of the sentence matches
# the length of the longest sentence
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '



In [12]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: hey how are yo
Target Sequence: ey how are you
Input Sequence: good i am fine
Target Sequence: ood i am fine 
Input Sequence: have a nice da
Target Sequence: ave a nice day


In [13]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

In [14]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [15]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

Input shape: (3, 14, 17) --> (Batch Size, Sequence Length, One-Hot Encoding Size)


In [16]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [17]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [18]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [19]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [20]:
# Training Run
input_seq = input_seq.to(device)
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq = input_seq.to(device)
    output, hidden = model(input_seq)
    output = output.to(device)
    target_seq = target_seq.to(device)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 2.3997
Epoch: 20/100............. Loss: 2.1246
Epoch: 30/100............. Loss: 1.7522
Epoch: 40/100............. Loss: 1.3222
Epoch: 50/100............. Loss: 0.9516
Epoch: 60/100............. Loss: 0.6581
Epoch: 70/100............. Loss: 0.4557
Epoch: 80/100............. Loss: 0.3161
Epoch: 90/100............. Loss: 0.2265
Epoch: 100/100............. Loss: 0.1687


In [21]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character = character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [22]:
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [23]:
sample(model, 15, 'good')

'good i am fine '

## Naive Translation

https://github.com/gabrielloye/RNN-walkthrough/blob/master/main.ipynb

In [63]:
import string
allchars = string.ascii_letters + string.digits + "_ ."
allchars

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_ .'

In [64]:
import torch
from torch import nn

import numpy as np

In [65]:
text = {'hey how are you': 'hey how are you', 'good i am fine': 'good i am fine', 'have a nice day': 'have a nice day'}
text = {'UserModel': 'UserGQLModel', 'GroupModel': 'GroupGQLModel', 'MembershipModel': 'MembershipGQLModel'}

# Join all the sentences together and extract the unique characters from the combined sentences
keys = list(text.keys())
values = list(text.values())

chars = set(''.join(keys).join(values))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(allchars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}
print(char2int)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, 'Z': 51, '0': 52, '1': 53, '2': 54, '3': 55, '4': 56, '5': 57, '6': 58, '7': 59, '8': 60, '9': 61, '_': 62, ' ': 63, '.': 64}


In [66]:
maxlen = len(max(keys + values, key=len))
print("The longest string has {} characters".format(maxlen))

The longest string has 18 characters


In [67]:


# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of the sentence matches
# the length of the longest sentence
for i in range(len(keys)):
    while len(keys[i])<maxlen:
        keys[i] += ' '

for i in range(len(values)):
    while len(values[i])<maxlen:
        values[i] += ' '

print(keys)
print(values)

['UserModel         ', 'GroupModel        ', 'MembershipModel   ']
['UserGQLModel      ', 'GroupGQLModel     ', 'MembershipGQLModel']


In [68]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(keys)):
    # Remove last character for input sequence
    input_seq.append(keys[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(values[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: UserModel        
Target Sequence: serGQLModel      
Input Sequence: GroupModel       
Target Sequence: roupGQLModel     
Input Sequence: MembershipModel  
Target Sequence: embershipGQLModel


In [69]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

In [70]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(keys)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [71]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))
#print(input_seq)

Input shape: (3, 17, 65) --> (Batch Size, Sequence Length, One-Hot Encoding Size)


In [72]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [81]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [82]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [83]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 10000
lr = 0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [84]:
# Training Run
input_seq = input_seq.to(device)
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq = input_seq.to(device)
    output, hidden = model(input_seq)
    output = output.to(device)
    target_seq = target_seq.to(device)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%100 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 100/10000............. Loss: 0.1300
Epoch: 200/10000............. Loss: 0.0198
Epoch: 300/10000............. Loss: 0.0094
Epoch: 400/10000............. Loss: 0.0057
Epoch: 500/10000............. Loss: 0.0039
Epoch: 600/10000............. Loss: 0.0029
Epoch: 700/10000............. Loss: 0.0023
Epoch: 800/10000............. Loss: 0.0018
Epoch: 900/10000............. Loss: 0.0015
Epoch: 1000/10000............. Loss: 0.0012
Epoch: 1100/10000............. Loss: 0.0010
Epoch: 1200/10000............. Loss: 0.0009
Epoch: 1300/10000............. Loss: 0.0008
Epoch: 1400/10000............. Loss: 0.0007
Epoch: 1500/10000............. Loss: 0.0006
Epoch: 1600/10000............. Loss: 0.0005
Epoch: 1700/10000............. Loss: 0.0005
Epoch: 1800/10000............. Loss: 0.0004
Epoch: 1900/10000............. Loss: 0.0004
Epoch: 2000/10000............. Loss: 0.0004
Epoch: 2100/10000............. Loss: 0.0003
Epoch: 2200/10000............. Loss: 0.0003
Epoch: 2300/10000............. Loss: 0.00

In [85]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character = character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [86]:
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [87]:
print(sample(model, 15, 'User'))
print(sample(model, 10, 'Group'))
print(sample(model, 20, 'Mem'))
print(sample(model, 20, 'UserModel'))

userGMeMeMeMeMe
groupGQLlu
membserGQLMupGQLMoem
usermodeldelsMemberG


https://www.youtube.com/watch?v=aOEoxyA4uXU

https://github.com/AssemblyAI-Examples/Machine-Learning-From-Scratch/tree/main